## ⚡️ How to train deberta to reach LB: 0.966
This is the training scriped I used to train the model used in my notebook ["DeBERTa-v3 Single Model LB:0.966"](https://www.kaggle.com/code/emiz6413/deberta-v3-single-model-lb-0-966)

#### Key points are as follows:
1. The (original) training data is splitted to 4 folds according to their `document % 4`

2. [MPWARE's dataset](https://www.kaggle.com/datasets/mpware/pii-mixtral8x7b-generated-essays) is added to each training set
I've tried different external datasets (though not very extensively) and found this works the best

3. The first 6 layers in the encoder are frozen while the embedding layer is trainable<br>

4. Training & evaluation dataset is truncated with `MAX_LENGTH=3072` <br>
My previous [notebook](https://www.kaggle.com/code/emiz6413/945-947-deberta-v3-base-infer-truncation-false) found longer input sequence inference performs better than striding inference, which made me think longer input sequence during training also improves the performance.

5. Evaluation is done every 50 steps and computes overall f5/recall/precision and entity-wise f5/recall/precision based on spacy token level prediction and ground-truth. The best checkpoint based on eval/f5 in each fold is selected as the final model

#### Note
I found by coincidence that thresholding on logits instead of softmax of logits can find better models on LB even though inference is done with thresholding on softmax. Please refer `MetricsComputer` for implementation. I haven't figured out why this is the case, so please let me know your thoughts in the comment.

In [1]:
import json
import copy
import gc
import os
import re
from collections import defaultdict
from pathlib import Path

import torch
from torch import nn
import numpy as np
import pandas as pd
from spacy.lang.en import English
from transformers.tokenization_utils import PreTrainedTokenizerBase
from transformers.models.deberta_v2 import DebertaV2ForTokenClassification, DebertaV2TokenizerFast
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers.trainer_utils import EvalPrediction
from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, concatenate_datasets
import wandb

2024-04-06 21:50:48.914674: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 21:50:48.942194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 21:50:49.448255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-06 21:50:49.924551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

## Config & Parameters

In [2]:
TRAINING_MODEL_PATH = "microsoft/deberta-v3-large"
# TRAINING_MAX_LENGTH = 3072
# EVAL_MAX_LENGTH = 3072
TRAINING_MAX_LENGTH = 4072
EVAL_MAX_LENGTH = 4072
CONF_THRESH = 0.9
LR = 2.5e-5
LR_SCHEDULER_TYPE = "linear"
NUM_EPOCHS = 5
BATCH_SIZE = 1
EVAL_BATCH_SIZE = 1
GRAD_ACCUMULATION_STEPS = 16 // BATCH_SIZE
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
AMP = True
FREEZE_EMBEDDING = False
FREEZE_LAYERS = 6
N_SPLITS = 1
NEGATIVE_RATIO = 0.3  # down sample ratio of negative samples in the training set
OUTPUT_DIR = "output"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

In [3]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=AMP,
    learning_rate=LR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    gradient_checkpointing=True,
    report_to="none",
    evaluation_strategy="steps",
    eval_steps=50,
    eval_delay=100,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=10,
    logging_steps=10,
    metric_for_best_model="f5",
    greater_is_better=True,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
)

## Dataset Preparation

In [4]:
with Path("input/pii-detection-removal-from-educational-data/train.json").open("r") as f:
    original_data = json.load(f)

with Path("input/pii-mixtral8x7b-generated-essays/mpware_mixtral8x7b_v1.1-no-i-username.json").open("r") as f:
    extra_data = json.load(f)
print("MPWARE's datapoints: ", len(extra_data))

MPWARE's datapoints:  2692


In [5]:
all_labels = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O'
]
id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

## Tokenization

In [6]:
class CustomTokenizer:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, label2id: dict, max_length: int) -> None:
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length

    def __call__(self, example: dict) -> dict:
        # rebuild text from tokens
        text, labels, token_map = [], [], []

        for idx, (t, l, ws) in enumerate(
            zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"])
        ):
            text.append(t)
            labels.extend([l] * len(t))
            token_map.extend([idx]*len(t))

            if ws:
                text.append(" ")
                labels.append("O")
                token_map.append(-1)

        text = "".join(text)
        labels = np.array(labels)

        # actual tokenization
        tokenized = self.tokenizer(
            "".join(text),
            return_offsets_mapping=True,
            truncation=True,
            max_length=self.max_length
        )

        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping:
            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(self.label2id["O"])
                continue

            # case when token starts with whitespace
            if text[start_idx].isspace():
                start_idx += 1

            token_labels.append(self.label2id[labels[start_idx]])

        length = len(tokenized.input_ids)

        return {**tokenized, "labels": token_labels, "length": length, "token_map": token_map}

## Instantiate the dataset

In [7]:
tokenizer = DebertaV2TokenizerFast.from_pretrained(TRAINING_MODEL_PATH)
train_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=TRAINING_MAX_LENGTH)
eval_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=EVAL_MAX_LENGTH)

ds = DatasetDict()

for key, data in zip(["original", "extra"], [original_data, extra_data]):
    ds[key] = Dataset.from_dict({
        "full_text": [x["full_text"] for x in data],
        "document": [str(x["document"]) for x in data],
        "tokens": [x["tokens"] for x in data],
        "trailing_whitespace": [x["trailing_whitespace"] for x in data],
        "provided_labels": [x["labels"] for x in data],
    })

/home/minori/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Metrics

In [8]:
def find_span(target: list[str], document: list[str]) -> list[list[int]]:
    idx = 0
    spans = []
    span = []

    for i, token in enumerate(document):
        if token != target[idx]:
            idx = 0
            span = []
            continue
        span.append(i)
        idx += 1
        if idx == len(target):
            spans.append(span)
            span = []
            idx = 0
            continue

    return spans


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}

In [9]:
class MetricsComputer:
    nlp = English()

    def __init__(self, eval_ds: Dataset, label2id: dict, conf_thresh: float = 0.9) -> None:
        self.ds = eval_ds.remove_columns("labels").rename_columns({"provided_labels": "labels"})
        self.gt_df = self.create_gt_df(self.ds)
        self.label2id = label2id
        self.confth = conf_thresh
        self._search_gt()

    def __call__(self, eval_preds: EvalPrediction) -> dict:
        pred_df = self.create_pred_df(eval_preds.predictions)
        return self.compute_metrics_from_df(self.gt_df, pred_df)

    def _search_gt(self) -> None:
        email_regex = re.compile(r'[\w.+-]+@[\w-]+\.[\w.-]+')
        phone_num_regex = re.compile(r"(\(\d{3}\)\d{3}\-\d{4}\w*|\d{3}\.\d{3}\.\d{4})\s")
        self.emails = []
        self.phone_nums = []

        for _data in self.ds:
            # email
            for token_idx, token in enumerate(_data["tokens"]):
                if re.fullmatch(email_regex, token) is not None:
                    self.emails.append(
                        {"document": _data["document"], "token": token_idx, "label": "B-EMAIL", "token_str": token}
                    )
            # phone number
            matches = phone_num_regex.findall(_data["full_text"])
            if not matches:
                continue
            for match in matches:
                target = [t.text for t in self.nlp.tokenizer(match)]
                matched_spans = find_span(target, _data["tokens"])
            for matched_span in matched_spans:
                for intermediate, token_idx in enumerate(matched_span):
                    prefix = "I" if intermediate else "B"
                    self.phone_nums.append(
                        {"document": _data["document"], "token": token_idx, "label": f"{prefix}-PHONE_NUM", "token_str": _data["tokens"][token_idx]}
                    )

    @staticmethod
    def create_gt_df(ds: Dataset):
        gt = []
        for row in ds:
            for token_idx, (token, label) in enumerate(zip(row["tokens"], row["labels"])):
                if label == "O":
                    continue
                gt.append(
                    {"document": row["document"], "token": token_idx, "label": label, "token_str": token}
                )
        gt_df = pd.DataFrame(gt)
        gt_df["row_id"] = gt_df.index

        return gt_df

    def create_pred_df(self, logits: np.ndarray) -> pd.DataFrame:
        """
        Note:
            Thresholing is doen on logits instead of softmax, which could find better models on LB.
        """
        prediction = logits
        o_index = self.label2id["O"]
        preds = prediction.argmax(-1)
        preds_without_o = prediction.copy()
        preds_without_o[:,:,o_index] = 0
        preds_without_o = preds_without_o.argmax(-1)
        o_preds = prediction[:,:,o_index]
        preds_final = np.where(o_preds < self.confth, preds_without_o , preds)

        pairs = set()
        processed = []

        # Iterate over document
        for p_doc, token_map, offsets, tokens, doc in zip(
            preds_final, self.ds["token_map"], self.ds["offset_mapping"], self.ds["tokens"], self.ds["document"]
        ):
            # Iterate over sequence
            for p_token, (start_idx, end_idx) in zip(p_doc, offsets):
                label_pred = id2label[p_token]

                if start_idx + end_idx == 0:
                    # [CLS] token i.e. BOS
                    continue

                if token_map[start_idx] == -1:
                    start_idx += 1

                # ignore "\n\n"
                while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                    start_idx += 1

                if start_idx >= len(token_map):
                    break

                token_id = token_map[start_idx]
                pair = (doc, token_id)

                # ignore "O", preds, phone number and  email
                if label_pred in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM") or token_id == -1:
                    continue

                if pair in pairs:
                    continue

                processed.append(
                    {"document": doc, "token": token_id, "label": label_pred, "token_str": tokens[token_id]}
                )
                pairs.add(pair)

        pred_df = pd.DataFrame(processed + self.emails + self.phone_nums)
        pred_df["row_id"] = list(range(len(pred_df)))

        return pred_df

    def compute_metrics_from_df(self, gt_df, pred_df):
        """
        Compute the LB metric (lb) and other auxiliary metrics
        """

        references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
        predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

        score_per_type = defaultdict(PRFScore)
        references = set(references)

        for ex in predictions:
            pred_type = ex[-1] # (document, token, label)
            if pred_type != 'O':
                pred_type = pred_type[2:] # avoid B- and I- prefix

            if pred_type not in score_per_type:
                score_per_type[pred_type] = PRFScore()

            if ex in references:
                score_per_type[pred_type].tp += 1
                references.remove(ex)
            else:
                score_per_type[pred_type].fp += 1

        for doc, tok, ref_type in references:
            if ref_type != 'O':
                ref_type = ref_type[2:] # avoid B- and I- prefix

            if ref_type not in score_per_type:
                score_per_type[ref_type] = PRFScore()
            score_per_type[ref_type].fn += 1

        totals = PRFScore()

        for prf in score_per_type.values():
            totals += prf

        return {
            "precision": totals.precision,
            "recall": totals.recall,
            "f5": totals.f5,
            **{
                f"{v_k}-{k}": v_v
                for k in set([l[2:] for l in self.label2id.keys() if l!= 'O'])
                for v_k, v_v in score_per_type[k].to_dict().items()
            },
        }

## Model

In [10]:
class ModelInit:
    def __init__(
        self,
        checkpoint: str,
        id2label: dict,
        label2id: dict,
        freeze_embedding: bool,
        freeze_layers: int,
    ) -> None:
        self.model = DebertaV2ForTokenClassification.from_pretrained(
            checkpoint,
            num_labels=len(id2label),
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
        for param in self.model.deberta.embeddings.parameters():
            param.requires_grad = False if freeze_embedding else True
        for layer in self.model.deberta.encoder.layer[:freeze_layers]:
            for param in layer.parameters():
                param.requires_grad = False
        self.weight = copy.deepcopy(self.model.state_dict())

    def __call__(self) -> DebertaV2ForTokenClassification:
        self.model.load_state_dict(self.weight)
        return self.model

model_init = ModelInit(
    TRAINING_MODEL_PATH,
    id2label=id2label,
    label2id=label2id,
    freeze_embedding=FREEZE_EMBEDDING,
    freeze_layers=FREEZE_LAYERS,
)

/home/minori/anaconda3/envs/torch/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Split 
Split the original dataset into 4 folds according to `document % 4` <br>
Only uses the first 30% of negative samples in the training set but they are NOT excluded from the eval set to make sure cross-evalidation is done on the entire training dataset.

In [11]:
# split according to document id
folds = [
    (
        np.array([i for i, d in enumerate(ds["original"]["document"]) if int(d) % N_SPLITS == s]),
        np.array([i for i, d in enumerate(ds["original"]["document"]) if int(d) % N_SPLITS != s]),
    )
    for s in range(N_SPLITS)
]

negative_idxs = [i for i, labels in enumerate(ds["original"]["provided_labels"]) if not any(np.array(labels) != "O")]
exclude_indices = negative_idxs[int(len(negative_idxs) * NEGATIVE_RATIO):]

In [12]:
folds

[(array([   0,    1,    2, ..., 6804, 6805, 6806]), array([], dtype=float64))]

## Train
Performs cross-validation and save the best checkpoint's metrics as json.

In [13]:
for fold_idx, (train_idx, eval_idx) in enumerate(folds):
    eval_idx = np.random.choice(train_idx, len(train_idx) // 6, replace=False)
    args.run_name = f"fold-{fold_idx}"
    args.output_dir = os.path.join(OUTPUT_DIR, f"fold_{fold_idx}")
    original_ds = ds["original"].select([i for i in train_idx if i not in exclude_indices])
    train_ds = concatenate_datasets([original_ds, ds["extra"]])
    train_ds = train_ds.map(train_encoder, num_proc=os.cpu_count())
    eval_ds = ds["original"].select(eval_idx)
    eval_ds = eval_ds.map(eval_encoder, num_proc=os.cpu_count())
    trainer = Trainer(
        args=args,
        model_init=model_init,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        compute_metrics=MetricsComputer(eval_ds=eval_ds, label2id=label2id),
        data_collator=DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16),
    )
    # break # delete this line to reproduce the result.
    trainer.train()
    eval_res = trainer.evaluate(eval_dataset=eval_ds)
    with open(os.path.join(args.output_dir, "eval_result.json"), "w") as f:
        json.dump(eval_res, f)
    del trainer
    gc.collect()
    torch.cuda.empty_cache()

Map (num_proc=12):   0%|          | 0/5395 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/1134 [00:00<?, ? examples/s]

  0%|          | 0/1685 [00:00<?, ?it/s]

{'loss': 2.4813, 'learning_rate': 1.3313609467455623e-06, 'epoch': 0.03}
{'loss': 2.0736, 'learning_rate': 2.8106508875739646e-06, 'epoch': 0.06}
{'loss': 1.0406, 'learning_rate': 4.2899408284023666e-06, 'epoch': 0.09}
{'loss': 0.2373, 'learning_rate': 5.76923076923077e-06, 'epoch': 0.12}
{'loss': 0.1023, 'learning_rate': 7.248520710059171e-06, 'epoch': 0.15}
{'loss': 0.0647, 'learning_rate': 8.727810650887574e-06, 'epoch': 0.18}
{'loss': 0.0413, 'learning_rate': 1.0207100591715976e-05, 'epoch': 0.21}
{'loss': 0.0303, 'learning_rate': 1.168639053254438e-05, 'epoch': 0.24}
{'loss': 0.0322, 'learning_rate': 1.3165680473372782e-05, 'epoch': 0.27}
{'loss': 0.017, 'learning_rate': 1.4644970414201184e-05, 'epoch': 0.3}


  0%|          | 0/1134 [00:00<?, ?it/s]

...

{'eval_loss': 0.0002648408408276737, 'eval_precision': 0.9188118811881189, 'eval_recall': 1.0, 'eval_f5': 0.9966129698471705, 'eval_p-STREET_ADDRESS': 0.0, 'eval_r-STREET_ADDRESS': 0.0, 'eval_f5-STREET_ADDRESS': 0.0, 'eval_p-USERNAME': 1.0, 'eval_r-USERNAME': 1.0, 'eval_f5-USERNAME': 1.0, 'eval_p-EMAIL': 1.0, 'eval_r-EMAIL': 1.0, 'eval_f5-EMAIL': 1.0, 'eval_p-ID_NUM': 1.0, 'eval_r-ID_NUM': 1.0, 'eval_f5-ID_NUM': 1.0, 'eval_p-NAME_STUDENT': 0.92, 'eval_r-NAME_STUDENT': 1.0, 'eval_f5-NAME_STUDENT': 0.9966666666666667, 'eval_p-URL_PERSONAL': 0.84375, 'eval_r-URL_PERSONAL': 1.0, 'eval_f5-URL_PERSONAL': 0.9929278642149929, 'eval_p-PHONE_NUM': 1.0, 'eval_r-PHONE_NUM': 1.0, 'eval_f5-PHONE_NUM': 1.0, 'eval_runtime': 34.7188, 'eval_samples_per_second': 32.662, 'eval_steps_per_second': 32.662, 'epoch': 3.26}
{'loss': 0.0004, 'learning_rate': 9.49868073878628e-06, 'epoch': 3.29}
{'loss': 0.0004, 'learning_rate': 9.33377308707124e-06, 'epoch': 3.32}
{'loss': 0.0004, 'learning_rate': 9.168865435356

  0%|          | 0/1134 [00:00<?, ?it/s]

{'eval_loss': 0.00038401922211050987, 'eval_precision': 0.8738229755178908, 'eval_recall': 1.0, 'eval_f5': 0.9944769598549171, 'eval_p-STREET_ADDRESS': 0.0, 'eval_r-STREET_ADDRESS': 0.0, 'eval_f5-STREET_ADDRESS': 0.0, 'eval_p-USERNAME': 1.0, 'eval_r-USERNAME': 1.0, 'eval_f5-USERNAME': 1.0, 'eval_p-EMAIL': 1.0, 'eval_r-EMAIL': 1.0, 'eval_f5-EMAIL': 1.0, 'eval_p-ID_NUM': 0.8571428571428571, 'eval_r-ID_NUM': 1.0, 'eval_f5-ID_NUM': 0.9936305732484076, 'eval_p-NAME_STUDENT': 0.8734177215189873, 'eval_r-NAME_STUDENT': 1.0, 'eval_f5-NAME_STUDENT': 0.9944567627494456, 'eval_p-URL_PERSONAL': 0.8181818181818182, 'eval_r-URL_PERSONAL': 1.0, 'eval_f5-URL_PERSONAL': 0.9915254237288135, 'eval_p-PHONE_NUM': 1.0, 'eval_r-PHONE_NUM': 1.0, 'eval_f5-PHONE_NUM': 1.0, 'eval_runtime': 34.797, 'eval_samples_per_second': 32.589, 'eval_steps_per_second': 32.589, 'epoch': 3.41}
{'loss': 0.0002, 'learning_rate': 8.674142480211083e-06, 'epoch': 3.44}
{'loss': 0.0002, 'learning_rate': 8.509234828496044e-06, 'epoch

  0%|          | 0/1134 [00:00<?, ?it/s]

{'eval_loss': 0.0003192301082890481, 'eval_precision': 0.8838095238095238, 'eval_recall': 1.0, 'eval_f5': 0.9949690721649485, 'eval_p-STREET_ADDRESS': 0.0, 'eval_r-STREET_ADDRESS': 0.0, 'eval_f5-STREET_ADDRESS': 0.0, 'eval_p-USERNAME': 1.0, 'eval_r-USERNAME': 1.0, 'eval_f5-USERNAME': 1.0, 'eval_p-EMAIL': 1.0, 'eval_r-EMAIL': 1.0, 'eval_f5-EMAIL': 1.0, 'eval_p-ID_NUM': 1.0, 'eval_r-ID_NUM': 1.0, 'eval_f5-ID_NUM': 1.0, 'eval_p-NAME_STUDENT': 0.8808510638297873, 'eval_r-NAME_STUDENT': 1.0, 'eval_f5-NAME_STUDENT': 0.9948243992606285, 'eval_p-URL_PERSONAL': 0.84375, 'eval_r-URL_PERSONAL': 1.0, 'eval_f5-URL_PERSONAL': 0.9929278642149929, 'eval_p-PHONE_NUM': 1.0, 'eval_r-PHONE_NUM': 1.0, 'eval_f5-PHONE_NUM': 1.0, 'eval_runtime': 34.7939, 'eval_samples_per_second': 32.592, 'eval_steps_per_second': 32.592, 'epoch': 3.56}


In [ ]:

# train_ds[70]['length']
for i in range(60, 75):
    print(train_ds[i]['length'])

595
928
916
520
813
703
1293
296
1555
880
459
601
781
682
1086


In [ ]:
train_ds[69]['length']


880